<a href="https://colab.research.google.com/github/yyinsomnia/build-cngpt/blob/main/play_gpt_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# init 环境和数据




In [3]:
# ! pip install transformers

# play embd

1. 首先还是要能learn的，不然就只有forward没有backward了

In [4]:
import torch

# 模拟一个batch的input
vocab_size = 3758  # 3500个常用字 + 256个ASCII + 2个[UNK] [MASK]
B, T = 8, 16

shape = (8, 16, 1) # (B, T, id)
token_ids = torch.randint(low=0, high=3758, size=shape)
print(token_ids)

tensor([[[2043],
         [2214],
         [1451],
         [ 579],
         [ 936],
         [2471],
         [ 613],
         [2677],
         [ 664],
         [ 306],
         [ 376],
         [3455],
         [3527],
         [3383],
         [3108],
         [ 764]],

        [[ 412],
         [1935],
         [ 415],
         [2292],
         [ 496],
         [3198],
         [2860],
         [ 714],
         [3599],
         [2369],
         [3043],
         [3444],
         [3077],
         [2359],
         [3374],
         [2564]],

        [[2548],
         [ 414],
         [ 839],
         [2303],
         [ 474],
         [3002],
         [ 984],
         [ 693],
         [1598],
         [ 335],
         [  42],
         [1794],
         [  21],
         [2100],
         [ 624],
         [ 425]],

        [[3013],
         [ 616],
         [3134],
         [3440],
         [1470],
         [3730],
         [2779],
         [2204],
         [3350],
         [ 121],
        

* 注意：这里的共3个维度，最后一维度也是向量。

In [5]:
# 第2个batch的第4个token，也是一个向量
print(token_ids[1, 3])

tensor([2292])


In [6]:
from torch import nn

embd_dim = 256

embd = nn.Embedding(vocab_size, embd_dim)

x = embd(token_ids)

print(x.shape) # (B, T, C) C就是embd_dim

torch.Size([8, 16, 1, 256])


发现shape和预期的不符合

pytorch的docs里面
Shape Input: (\*) Output: (\*, H)


In [7]:
x

tensor([[[[ 1.1412,  1.4311, -1.5662,  ..., -0.9454,  0.7590,  1.8491]],

         [[-1.3643, -1.7146,  0.9686,  ..., -0.4602,  2.1885,  0.7165]],

         [[ 0.0306, -1.4302,  3.0268,  ...,  0.0419,  0.8861, -0.5051]],

         ...,

         [[-0.0506, -0.6311, -1.1254,  ..., -0.2504, -0.2527,  0.0973]],

         [[-0.9731,  0.6187,  1.3248,  ..., -1.3551, -1.4239, -0.4659]],

         [[-0.5806,  1.2208,  0.7113,  ...,  1.8048,  1.2067, -1.8959]]],


        [[[ 0.7007, -0.4798,  0.8899,  ...,  0.9325,  0.0791,  1.3326]],

         [[ 1.2629,  1.8462, -0.7314,  ..., -0.5244, -0.5179,  0.8159]],

         [[-1.5975, -0.2385, -0.3092,  ...,  0.6572, -0.5316, -0.8374]],

         ...,

         [[ 1.3551, -0.1080, -0.6651,  ...,  1.2770,  0.6605,  0.0426]],

         [[-0.7347,  0.4656,  1.4997,  ...,  0.8134, -0.3868, -0.1720]],

         [[-0.5680,  1.4662,  0.3667,  ...,  0.1219, -0.8770, -0.9035]]],


        [[[-0.7831,  0.2401, -0.7243,  ..., -1.1168, -0.5934,  0.0601]],

    

仔细想一下是合理的。如果是图片，那有RGB 3个值，就变成了(B, H, W, 3)
这样RGB每个过完embd，应该是变成了(B, H, W, 3, embd_dim)

In [8]:
# 简单一点把最后一维弄成标量看看
shape = (8, 16,)
token_ids = torch.randint(low=0, high=vocab_size, size=shape)
print(token_ids)
# 这个输出是熟悉的感觉

tensor([[2030, 1191, 3354,  987, 3748,  256, 2650,  828, 2914, 3517,  702, 2666,
         3247, 2473, 3516, 2824],
        [3755, 1143, 2170, 1840, 3005, 1639, 1853, 2184, 3026,  149, 2665, 1055,
          406, 3474, 1240, 2015],
        [1387,   62, 1356, 2730, 3000, 1471,  554,  399, 3320, 3449, 3220, 2496,
           26, 2576, 1801, 2093],
        [2203, 2671, 2455, 1684,  718, 3288, 2779, 3680, 1265, 2196,  578,   27,
         1589, 1527,   64,  299],
        [  10, 2507,  966, 1497, 1635,  648, 1586, 2647,  718, 3235, 1241, 3553,
         2156, 1846, 2309, 1776],
        [3312, 2619, 3611, 3591, 1380, 1427, 1549, 2403, 1998, 2006, 1904,  551,
         3506, 3179, 3065, 1833],
        [2455, 3247, 2880, 2834, 1317, 2563,  130, 1604,    6, 3296,  405, 3257,
         3079, 3372,  980, 2699],
        [ 677,  196, 2449, 2717, 2366, 3295,  905, 1281, 1359, 1644,  210, 1647,
         3082,  317,   77, 3013]])


In [9]:
x = embd(token_ids)

print(x.shape)

# 这次感觉对了

torch.Size([8, 16, 256])


In [10]:
x
# 观察了一下，这个值还是有点技巧的，大部分都在 += 1.0 左右

tensor([[[ 1.0483e+00, -3.7320e-01,  9.5842e-01,  ..., -6.3092e-01,
          -5.0265e-01,  7.7691e-01],
         [-1.2889e+00, -7.0146e-01,  1.9759e+00,  ...,  6.2977e-01,
          -1.8848e+00, -1.9642e-01],
         [-4.2325e-01, -9.0744e-01,  1.3034e+00,  ..., -1.1252e+00,
          -2.8621e-01,  1.1301e-01],
         ...,
         [ 1.5173e+00,  1.4634e+00, -8.7972e-01,  ..., -2.6073e-02,
           3.1359e-01, -1.2791e+00],
         [-1.0178e+00, -5.6138e-01, -8.7657e-01,  ...,  8.1380e-01,
           1.5480e-01, -3.5459e-01],
         [ 4.3281e-01,  4.3787e-01,  3.1485e-01,  ...,  6.9691e-02,
          -7.6486e-01, -3.8529e-01]],

        [[ 1.3056e+00, -2.4012e-01, -1.9810e-01,  ...,  5.0410e-01,
           4.5134e-01, -9.9609e-01],
         [-2.6841e-02,  6.1632e-01, -2.0286e+00,  ...,  6.1581e-01,
          -6.1157e-02, -1.0268e+00],
         [-2.0581e-01,  2.2139e+00, -3.2843e-01,  ..., -5.8461e-01,
           1.0483e-02,  1.5929e+00],
         ...,
         [-4.8183e-01, -5

# play attention
关键词：**QKV** , **wei matrix**, **tril**, **softmax**

对matrix的理解还有不够深刻，需要组织一段语言或者一个动图

In [11]:
headers_num = 8
header_dim = embd_dim // headers_num

query = nn.Linear(embd_dim, header_dim)
key = nn.Linear(embd_dim, header_dim)
value = nn.Linear(embd_dim, header_dim)

q = query(x) # shape(B, T, header_dim) = (8, 16, 32)
k = key(x)
v = value(x)

print(q.shape)

torch.Size([8, 16, 32])


In [12]:
import torch.nn.functional as F

wei = q @ k.transpose(-2, -1)
print(wei.shape) # shape(B, T, T)

tril = torch.tril(torch.ones(T, T))
print(tril == 0)

wei = wei.masked_fill(tril == 0, float('-inf'))

# print(wei)

wei = F.softmax(wei, dim=-1)
print(wei)

x_attn = wei @ v

torch.Size([8, 16, 16])
tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True],
        [False, False, False, False, False,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False,  True,  True,  True,
          True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, F

# play MHA
- 思路1: 肉写,比较机械 key1,query1,value1, key2,query2,value2...
- 思路2: flatten一下，key_0to7。nn.Linear(embd_dim, header_dim * 8)
- 思路3: nn.ModuleList(),印象中Andrej是用的这个，但是具体怎么用还没想好
- 思路4: 之前问ChatGPT的实现，应该比较高效，是把多头的参数直接cat起来的

In [ ]:
# 思路1
query1 = nn.Linear(embd_dim, header_dim)
key1 = nn.Linear(embd_dim, header_dim)
value1 = nn.Linear(embd_dim, header_dim)

query2 = nn.Linear(embd_dim, header_dim)
key2 = nn.Linear(embd_dim, header_dim)
value2 = nn.Linear(embd_dim, header_dim)

query3 = nn.Linear(embd_dim, header_dim)
key3 = nn.Linear(embd_dim, header_dim)
value3 = nn.Linear(embd_dim, header_dim)

query4 = nn.Linear(embd_dim, header_dim)
key4 = nn.Linear(embd_dim, header_dim)
value4 = nn.Linear(embd_dim, header_dim)

query5 = nn.Linear(embd_dim, header_dim)
key5 = nn.Linear(embd_dim, header_dim)
value5 = nn.Linear(embd_dim, header_dim)

query6 = nn.Linear(embd_dim, header_dim)
key6 = nn.Linear(embd_dim, header_dim)
value6 = nn.Linear(embd_dim, header_dim)

query7 = nn.Linear(embd_dim, header_dim)
key7 = nn.Linear(embd_dim, header_dim)
value7 = nn.Linear(embd_dim, header_dim)

query8 = nn.Linear(embd_dim, header_dim)
key8 = nn.Linear(embd_dim, header_dim)
value8 = nn.Linear(embd_dim, header_dim)

q1 = query1(x)
k1 = key1(x)
v1 = value1(x)

q2 = query2(x)
k2 = key2(x)
v2 = value2(x)

q3 = query3(x)
k3 = key3(x)
v3 = value3(x)

q4 = query4(x)
k4 = key4(x)
v4 = value4(x)

q5 = query5(x)
k5 = key5(x)
v5 = value5(x)

q6 = query6(x)
k6 = key6(x)
v6 = value6(x)

q7 = query7(x)
k7 = key7(x)
v7 = value7(x)

q8 = query8(x)
k8 = key8(x)
v8 = value8(x)

tril = torch.tril(torch.ones(T, T))

wei1 = q1 @ k1.transpose(-2, -1)
wei1 = wei1.masked_fill(tril == 0, float('-inf'))
wei1 = F.softmax(wei1, dim=-1)
x_header_1 = wei1 @ v1

wei2 = q2 @ k2.transpose(-2, -1)
wei2 = wei2.masked_fill(tril == 0, float('-inf'))
wei2 = F.softmax(wei2, dim=-1)
x_header_2 = wei2 @ v2

wei3 = q3 @ k3.transpose(-2, -1)
wei3 = wei3.masked_fill(tril == 0, float('-inf'))
wei3 = F.softmax(wei3, dim=-1)
x_header_3 = wei3 @ v3

wei4 = q4 @ k4.transpose(-2, -1)
wei4 = wei4.masked_fill(tril == 0, float('-inf'))
wei4 = F.softmax(wei4, dim=-1)
x_header_4 = wei4 @ v4

wei5 = q5 @ k5.transpose(-2, -1)
wei5 = wei5.masked_fill(tril == 0, float('-inf'))
wei5 = F.softmax(wei5, dim=-1)
x_header_5 = wei5 @ v5

wei6 = q6 @ k6.transpose(-2, -1)
wei6 = wei6.masked_fill(tril == 0, float('-inf'))
wei6 = F.softmax(wei6, dim=-1)
x_header_6 = wei6 @ v6

wei7 = q7 @ k7.transpose(-2, -1)
wei7 = wei7.masked_fill(tril == 0, float('-inf'))
wei7 = F.softmax(wei7, dim=-1)
x_header_7 = wei7 @ v7

wei8 = q8 @ k8.transpose(-2, -1)
wei8 = wei2.masked_fill(tril == 0, float('-inf'))
wei8 = F.softmax(wei8, dim=-1)
x_header_8 = wei8 @ v8 # shape(B, T, header_dim)

# 最后怎么合成到embd_dim，继续过FFN?
# shape(B, T, header_dim)，可以不考虑前面B, T 然后想象8个attn concat起来，过一个FC
# 这个等价于8个attn单独过一个自己的FC，然后再elementwise的add
# 所以继续发扬暴力写法。。。

fc1 = nn.Linear(header_dim, embd_dim)
fc2 = nn.Linear(header_dim, embd_dim)
fc3 = nn.Linear(header_dim, embd_dim)
fc4 = nn.Linear(header_dim, embd_dim)
fc5 = nn.Linear(header_dim, embd_dim)
fc6 = nn.Linear(header_dim, embd_dim)
fc7 = nn.Linear(header_dim, embd_dim)
fc8 = nn.Linear(header_dim, embd_dim)

x_mht = fc1(x_header_1) + fc2(x_header_2) + fc3(x_header_3) + fc4(x_header_4) + fc5(x_header_5) + fc6(x_header_6) + fc7(x_header_7) + fc8(x_header_8)

print(x_mht.shape) # shape=(8, 16, 256)

torch.Size([8, 16, 256])


* ChatGPT提到当模型结构较为复杂，需要灵活地管理子模块而不是简单地按顺序执行时，nn.ModuleList 提供了更大的灵活性。

In [ ]:
# 思路3，自己从头写

query_list = nn.ModuleList([nn.Linear(embd_dim, header_dim)] * 8)
key_list = nn.ModuleList([nn.Linear(embd_dim, header_dim)] * 8)
value_list = nn.ModuleList([nn.Linear(embd_dim, header_dim)] * 8)

q_list = [query(x) for query in query_list]
k_list = [key(x) for key in key_list]
v_list = [value(x) for value in value_list]

x_header_list = list()
for n in range(8):
    wei = q_list[n] @ k_list[n].transpose(-2, -1)
    wei = wei.masked_fill(tril == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    x_header = wei @ v[n]
    x_header_list.append(x_header)

fc_list = nn.ModuleList([nn.Linear(header_dim, embd_dim)] * 8)

x_mht = torch.zeros(8, 16, embd_dim)
for n in range(8):
    fc = fc_list[n]
    x_mht += fc(x_header_list[n])

print(x_mht.shape)

torch.Size([8, 16, 256])


* 上面这种写法，感觉并行度不够，多头都是串行的。看看标准答案吧，哈哈。
* 标准答案思路：Header和MultiHeader 2个类，然后ModuleList 进来，循环过完再cat+proj
* 标准答案其实也有串行的部分！另外标准答案里面还有个dropout，这块之前没注意到
* 另外还有一个，忘了除以根号d, 原因上节课有讲，是为了保证vars在1.0左右，不至于让softmax太peak变成了hardmax哈哈

In [ ]:
# 思路3，扫了下标准答案后
class Header(nn.Module):
    def __init__(self):
        super().__init__()
        self.query = nn.Linear(embd_dim, header_dim)
        self.key = nn.Linear(embd_dim, header_dim)
        self.value = nn.Linear(embd_dim, header_dim)

    def forward(self, x):
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        wei = q @ k.transpose(-2, -1)

        # 标准答案
        # wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)

        tril = torch.tril(torch.ones(T, T))
        wei = wei.masked_fill(tril == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)

        # 标准答案
        # wei = self.dropout(wei)

        x_attn = wei @ v

        return x_attn

class MultiHead(nn.ModuleList):
    def __init__(self, headers_num=8):
        super().__init__()

        self.headers = nn.ModuleList([Header()] * headers_num)
        self.proj = nn.Linear(header_dim * headers_num, embd_dim)
        # 标准答案
        # self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x_header_list = [h(x) for h in self.headers] # shape (n, B, T, header_dim)
        x_headers_flatten = torch.cat(x_header_list, dim=-1)
        x_mha = self.proj(x_headers_flatten)

        return x_mha


multihead = MultiHead()
x_mha = multihead(x)

print(x_mha.shape)

torch.Size([8, 16, 256])


使用在gpt-mao的代码，对比一下标准答案，有如下分析：
1. **参数名更标准：** 标准答案参数名叫c_attn，这个是为了和Transformers库保持一致方便load模型
2. **更高效：** 标准答案使用了nn.Linear(config.n_embd, 3 * config.n_embd)，一次搞定qkv。
3. **遵循原论文：** shape从(B, T, C) -> (B, T, n_head, C // n_head) 保证了 C = n_head * head_size。 Think: 为啥要保证这个？
4. **核心逻辑一致：**  (B, T, n_head, head_size) -> (B, n_head, T, head_size)。Think: 为什么这么处理还是没特别想透

In [13]:
# 思路4:

class GPTConfig:
    def __init__(self, config):
        self.B = config['B']
        self.T = config['T']
        self.C = config['C']

        self.vocab_size = config['vocab_size']

        self.header_size = config['header_size']
        self.headers_num = config['headers_num']

        self.blocks_num = config['blocks_num']
        self.ffn_size = config['ffn_size']

class MultiHeadV4(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.key = nn.Linear(config.C, config.header_size * config.headers_num)
        self.query = nn.Linear(config.C, config.header_size * config.headers_num)
        self.value = nn.Linear(config.C, config.header_size * config.headers_num)
        self.attn_fc = nn.Linear(config.header_size * config.headers_num, config.C)


    def split_headers(self, x, batch_size):
        """Split the last dimension into (num_heads, head_dim) and transpose the result."""
        x = x.view(batch_size, config.T, self.config.headers_num, self.config.header_size)

        return x.transpose(1, 2) # shape (B, headers_num, T, header_size) 这里需要thinking一下why

    def combine_headers(self, x, batch_size):
        """
            Combine the heads and restore the original last dimension.
            x.shape (B, headers_num, T, header_size)
        """
        x = x.transpose(1, 2).contiguous()
        # 这里view中的-1，表示自动计算。其实可以直接用T
        return x.view(batch_size, self.config.T, self.config.headers_num * self.config.header_size)

    def attn(self, x):
        """
        x.shape (B, T, C)
        """
        config = self.config
        B, T, C = x.shape

        # k, q, v shape (B, headers_num, T, header_size)
        k = self.split_headers(self.key(x), B)
        q = self.split_headers(self.query(x), B)
        v = self.split_headers(self.value(x), B)

        tril = torch.tril(torch.ones(config.T, config.T))
        wei = q @ k.transpose(-2, -1) # 这个时候用负数的兼容性就体现了
        wei = wei.masked_fill(tril == 0, float('-inf'))
        wei = wei / (config.header_size ** 0.5)
        wei = F.softmax(wei, dim=-1)
        # wei.shape (B, headers_num, T, T)

        out = wei @ v # out.shape (B, headers_num, T, header_size)
        out = self.combine_headers(out, B)  # out.shape(B, T, config.header_size * config.headers_num)
        x = self.attn_fc(out)

        return x


    def forward(self, x):
        x = self.attn(x)
        return x


config = GPTConfig({'B': B, 'T': T, 'C': embd_dim, 'vocab_size': vocab_size, 'header_size': header_dim, 'headers_num': headers_num, 'blocks_num': 4, 'ffn_size': 256})
mha = MultiHeadV4(config)

x_mha_v4 = mha(x)
print(x_mha_v4.shape)

torch.Size([8, 16, 256])
